# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [140]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col, isnull, isnan, year, month, day, dayofmonth, concat, dayofweek, dayofyear, weekofyear, weekday, hour, minute, second

In [2]:
spark = SparkSession.builder.appName('customer_churn').getOrCreate()
spark

In [3]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
df.count()

900

In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [6]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [25]:
df.drop('Onboard_date').show()

+-------------------+----+--------------+---------------+-----+---------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|13120 Daniel Moun...|           Smith Inc|    1|
|     Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|765 Tricia Row Ka...|          Love-Jones|    1|
|   Jessica Williams|48.

In [32]:
data_types = dict(df.dtypes)
data_types

{'Names': 'string',
 'Age': 'double',
 'Total_Purchase': 'double',
 'Account_Manager': 'int',
 'Years': 'double',
 'Num_Sites': 'double',
 'Onboard_date': 'timestamp',
 'Location': 'string',
 'Company': 'string',
 'Churn': 'int'}

In [33]:
numerical_types = {"int", "bigint", "float", "double", "decimal", "long"}
numerical_columns = [col_name for col_name, col_type in data_types.items() if col_type in numerical_types]
numerical_columns

['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Churn']

In [35]:
categorical_cols = [col_name for col_name, col_type in data_types.items() if col_type == 'string']
categorical_cols

['Names', 'Location', 'Company']

In [34]:
null_cols = [column for column in numerical_columns if df.filter(col(column).isNull() | isnan(col(column))).count() > 0]
null_cols

[]

In [36]:
null_cols = [column for column in categorical_cols if df.filter(col(column).isNull() | isnan(col(column))).count() > 0]
null_cols

[]

In [38]:
df.filter(col('Onboard_date').isNull()).count()

0

So, there are no null values in the dataset.

In [54]:
final_df = df.dropDuplicates()
final_df = final_df.dropna()
final_df.count()

900

In [55]:
final_df = final_df.withColumn('Year',year('Onboard_date'))
final_df = final_df.withColumn('Month',month('Onboard_date'))
final_df = final_df.withColumn('Day',day('Onboard_date'))
final_df = final_df.withColumn('Weekday',weekday('Onboard_date'))
final_df = final_df.withColumn('Week of Year',weekofyear('Onboard_date'))
final_df = final_df.withColumn('Day of Month',dayofmonth('Onboard_date'))
final_df = final_df.withColumn('Day of Week',dayofweek('Onboard_date'))
final_df = final_df.withColumn('Day of Year',dayofyear('Onboard_date'))
final_df = final_df.withColumn('Hour',hour('Onboard_date'))
final_df = final_df.withColumn('Minute',minute('Onboard_date'))
final_df = final_df.withColumn('Second',second('Onboard_date'))
final_df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'Week of Year',
 'Day of Month',
 'Day of Week',
 'Day of Year',
 'Hour',
 'Minute',
 'Second']

In [56]:
final_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Weekday: integer (nullable = true)
 |-- Week of Year: integer (nullable = true)
 |-- Day of Month: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Day of Year: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- Second: integer (nullable = true)



In [59]:
final_df.groupby('Company').count().show()

+--------------------+-----+
|             Company|count|
+--------------------+-----+
|            Soto PLC|    2|
|            Todd LLC|    1|
|           Smith PLC|    1|
|          Obrien PLC|    1|
|          Hall Group|    1|
|Hunter, Reyes and...|    1|
|Smith, Marshall a...|    1|
|Miller, Johnson a...|    1|
|       Smith-Carroll|    1|
|          Cannon Inc|    1|
|Hall, Hernandez a...|    1|
|Freeman, Lam and ...|    1|
|        Robinson PLC|    1|
|Wilson, Collins a...|    1|
|Jennings, Gates a...|    1|
|     Campbell-Willis|    1|
|    Martinez-Roberts|    1|
|        White-Dennis|    1|
|      Jackson-Garcia|    1|
|          Hayes-Hill|    1|
+--------------------+-----+
only showing top 20 rows



In [61]:
final_df = final_df.withColumnRenamed(existing='Years',new='Total_Years')
final_df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Total_Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'Week of Year',
 'Day of Month',
 'Day of Week',
 'Day of Year',
 'Hour',
 'Minute',
 'Second']

In [73]:
cols = ['Age','Total_Purchase','Account_Manager','Total_Years','Num_Sites','Year','Month','Day','Weekday','Week of Year','Day of Month','Day of Week','Day of Year','Hour','Minute','Second','Churn']

final_df = final_df.select(cols)
final_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Total_Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Weekday: integer (nullable = true)
 |-- Week of Year: integer (nullable = true)
 |-- Day of Month: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Day of Year: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- Second: integer (nullable = true)
 |-- Churn: integer (nullable = true)



In [74]:
final_df = final_df.na.drop()
final_df.count()

900

In [77]:
final_df.columns

['Age',
 'Total_Purchase',
 'Account_Manager',
 'Total_Years',
 'Num_Sites',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'Week of Year',
 'Day of Month',
 'Day of Week',
 'Day of Year',
 'Hour',
 'Minute',
 'Second',
 'Churn']

In [78]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Total_Years',
 'Num_Sites',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'Week of Year',
 'Day of Month',
 'Day of Week',
 'Day of Year',
 'Hour',
 'Minute',
 'Second'],outputCol='features')
assembler

VectorAssembler_146745618628

In [84]:
final_data = assembler.transform(final_df)
final_data

DataFrame[Age: double, Total_Purchase: double, Account_Manager: int, Total_Years: double, Num_Sites: double, Year: int, Month: int, Day: int, Weekday: int, Week of Year: int, Day of Month: int, Day of Week: int, Day of Year: int, Hour: int, Minute: int, Second: int, Churn: int, features: vector]

In [85]:
final_data.select('features','Churn').show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[45.0,12309.39,1....|    1|
|[38.0,8825.87,1.0...|    1|
|[39.0,6351.79,0.0...|    1|
|[33.0,10309.71,1....|    0|
|[39.0,11490.75,0....|    0|
|[44.0,11794.91,0....|    0|
|[42.0,9091.99,0.0...|    0|
|[41.0,6204.86,1.0...|    0|
|[32.0,8617.98,1.0...|    1|
|[33.0,12638.51,1....|    0|
|[46.0,8603.6,0.0,...|    0|
|[37.0,9499.3,1.0,...|    0|
|[29.0,11274.46,1....|    0|
|[33.0,10306.21,1....|    0|
|[60.0,9621.04,1.0...|    0|
|[39.0,12019.93,0....|    0|
|[53.0,9655.25,0.0...|    0|
|[39.0,5846.65,1.0...|    0|
|[45.0,11599.27,1....|    1|
|[48.0,6495.01,1.0...|    1|
+--------------------+-----+
only showing top 20 rows



In [86]:
final_data.count()

900

In [87]:
train_df, test_df = final_data.randomSplit([0.7,0.3])

In [88]:
lr = LogisticRegression(featuresCol='features',labelCol='Churn')
lr_model = lr.fit(train_df)

In [89]:
lr_model.summary

In [90]:
test_results = lr_model.evaluate(test_df)
test_results

In [91]:
test_results.accuracy

0.915057915057915

In [92]:
test_results.areaUnderROC

0.902054794520547

In [94]:
test_results.precisionByLabel

[0.933920704845815, 0.78125]

In [95]:
test_results.recallByLabel

[0.9680365296803652, 0.625]

In [97]:
test_results.fMeasureByLabel()

[0.9506726457399104, 0.6944444444444444]

In [98]:
test_results.weightedRecall

0.9150579150579151

In [99]:
test_results.weightedPrecision

0.9103422176109401

In [101]:
test_results.weightedFMeasure()

0.9111007227599157

In [102]:
test_results.weightedTruePositiveRate

0.9150579150579151

In [103]:
test_results.weightedFalsePositiveRate

0.32202138537754976

In [106]:
test_results.roc.show()

+------------------+-----+
|               FPR|  TPR|
+------------------+-----+
|               0.0|  0.0|
|               0.0|0.025|
|               0.0| 0.05|
|               0.0|0.075|
|               0.0|  0.1|
|               0.0|0.125|
|               0.0| 0.15|
|               0.0|0.175|
|               0.0|  0.2|
|               0.0|0.225|
|               0.0| 0.25|
|0.0045662100456621| 0.25|
|0.0091324200913242| 0.25|
|0.0091324200913242|0.275|
|0.0091324200913242|  0.3|
|0.0091324200913242|0.325|
|0.0136986301369863|0.325|
|0.0136986301369863| 0.35|
|0.0136986301369863|0.375|
|0.0182648401826484|0.375|
+------------------+-----+
only showing top 20 rows



In [107]:
test_results.pr.show()

+------+------------------+
|recall|         precision|
+------+------------------+
|   0.0|               1.0|
| 0.025|               1.0|
|  0.05|               1.0|
| 0.075|               1.0|
|   0.1|               1.0|
| 0.125|               1.0|
|  0.15|               1.0|
| 0.175|               1.0|
|   0.2|               1.0|
| 0.225|               1.0|
|  0.25|               1.0|
|  0.25|0.9090909090909091|
|  0.25|0.8333333333333334|
| 0.275|0.8461538461538461|
|   0.3|0.8571428571428571|
| 0.325|0.8666666666666667|
| 0.325|            0.8125|
|  0.35|0.8235294117647058|
| 0.375|0.8333333333333334|
| 0.375|0.7894736842105263|
+------+------------------+
only showing top 20 rows



In [108]:
test_results.predictions.show()

+----+--------------+---------------+-----------+---------+----+-----+---+-------+------------+------------+-----------+-----------+----+------+------+-----+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Account_Manager|Total_Years|Num_Sites|Year|Month|Day|Weekday|Week of Year|Day of Month|Day of Week|Day of Year|Hour|Minute|Second|Churn|            features|       rawPrediction|         probability|prediction|
+----+--------------+---------------+-----------+---------+----+-----+---+-------+------------+------------+-----------+-----------+----+------+------+-----+--------------------+--------------------+--------------------+----------+
|22.0|      11254.38|              1|       4.96|      8.0|2012|    6| 23|      5|          25|          23|          7|        175|   8|    40|    29|    0|[22.0,11254.38,1....|[4.75126142947075...|[0.99143323502473...|       0.0|
|25.0|       9672.03|              0|       5.49|      8.0|2006|    1|  

In [110]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')
evaluator

BinaryClassificationEvaluator_a10aa7b6853a

In [111]:
area_under_roc = evaluator.evaluate(test_results.predictions)
area_under_roc

0.7965182648401826

In [112]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn',metricName='areaUnderPR')
area_under_pr = evaluator.evaluate(test_results.predictions)
area_under_pr

0.663723153957529

In [156]:
unlabeled_data = spark.read.csv('new_customers.csv',inferSchema=True,header=True)
unlabeled_data.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [157]:
unlabeled_data.count()

6

In [158]:
unlabeled_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [159]:
unlabeled_data = unlabeled_data.withColumn('Year',year('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Month',month('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Day',day('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Weekday',weekday('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Week of Year',weekofyear('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Day of Month',dayofmonth('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Day of Week',dayofweek('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Day of Year',dayofyear('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Hour',hour('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Minute',minute('Onboard_date'))
unlabeled_data = unlabeled_data.withColumn('Second',second('Onboard_date'))
unlabeled_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'Week of Year',
 'Day of Month',
 'Day of Week',
 'Day of Year',
 'Hour',
 'Minute',
 'Second']

In [160]:
unlabeled_data = unlabeled_data.withColumnRenamed(existing='Years',new='Total_Years')

In [134]:
# unlabeled_data = unlabeled_data.select(['Age',
#  'Total_Purchase',
#  'Account_Manager',
#  'Total_Years',
#  'Num_Sites',
#  'Year',
#  'Month',
#  'Day',
#  'Weekday',
#  'Week of Year',
#  'Day of Month',
#  'Day of Week',
#  'Day of Year',
#  'Hour',
#  'Minute',
#  'Second'])
# unlabeled_data.columns

['Age',
 'Total_Purchase',
 'Account_Manager',
 'Total_Years',
 'Num_Sites',
 'Year',
 'Month',
 'Day',
 'Weekday',
 'Week of Year',
 'Day of Month',
 'Day of Week',
 'Day of Year',
 'Hour',
 'Minute',
 'Second']

In [161]:
unlabeled_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Total_Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Weekday: integer (nullable = true)
 |-- Week of Year: integer (nullable = true)
 |-- Day of Month: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Day of Year: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- Second: integer (nullable = true)



In [162]:
final_unlabeled_data = assembler.transform(unlabeled_data)
final_unlabeled_data.show()

+--------------+----+--------------+---------------+-----------+---------+-------------------+--------------------+----------------+----+-----+---+-------+------------+------------+-----------+-----------+----+------+------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Total_Years|Num_Sites|       Onboard_date|            Location|         Company|Year|Month|Day|Weekday|Week of Year|Day of Month|Day of Week|Day of Year|Hour|Minute|Second|            features|
+--------------+----+--------------+---------------+-----------+---------+-------------------+--------------------+----------------+----+-----+---+-------+------------+------------+-----------+-----------+----+------+------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1|       7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|2011|    8| 29|      0|          35|          29|          2|        241|  18|    37|    54|[37.0,9935.53,1.0...|
|Michele Wright|

In [163]:
test_predictions = lr_model.transform(final_unlabeled_data)
test_predictions

DataFrame[Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Total_Years: double, Num_Sites: double, Onboard_date: timestamp, Location: string, Company: string, Year: int, Month: int, Day: int, Weekday: int, Week of Year: int, Day of Month: int, Day of Week: int, Day of Year: int, Hour: int, Minute: int, Second: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [164]:
test_predictions.show()

+--------------+----+--------------+---------------+-----------+---------+-------------------+--------------------+----------------+----+-----+---+-------+------------+------------+-----------+-----------+----+------+------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Total_Years|Num_Sites|       Onboard_date|            Location|         Company|Year|Month|Day|Weekday|Week of Year|Day of Month|Day of Week|Day of Year|Hour|Minute|Second|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----------+---------+-------------------+--------------------+----------------+----+-----+---+-------+------------+------------+-----------+-----------+----+------+------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1|       7.71|      8.0|2011-08-29 18:37:54|38612 Joh

In [165]:
test_predictions.select(['Names','Company','prediction']).show()

+--------------+----------------+----------+
|         Names|         Company|prediction|
+--------------+----------------+----------+
| Andrew Mccall|        King Ltd|       0.0|
|Michele Wright|   Cannon-Benson|       1.0|
|  Jeremy Chang|Barron-Robertson|       1.0|
|Megan Ferguson|   Sexton-Golden|       1.0|
|  Taylor Young|        Wood LLC|       0.0|
| Jessica Drake|   Parks-Robbins|       1.0|
+--------------+----------------+----------+

